<a href="https://colab.research.google.com/github/makhijakabir/machine-learning/blob/main/bayesClassifierESFtraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training the datset using the bayes classifier to build an email spam filter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Imports

In [3]:
import numpy as np
import pandas as pd

##Constants

In [67]:
trainingDataFile = '/content/drive/MyDrive/Colab Notebooks/SpamData/01_Processing/train-data.txt'
testingDataFile = '/content/drive/MyDrive/Colab Notebooks/SpamData/01_Processing/test-data.txt'

tokenSpamProbFile = '/content/drive/MyDrive/Colab Notebooks/SpamData/03_Testing/prob-spam.txt'
tokeNotSpamProbFile = '/content/drive/MyDrive/Colab Notebooks/SpamData/03_Testing/prob-notspam.txt'

tokenAllProbFile = '/content/drive/MyDrive/Colab Notebooks/SpamData/03_Testing/prob-all-tokens.txt'

testFeaturesMatrixFile = '/content/drive/MyDrive/Colab Notebooks/SpamData/03_Testing/test-features.txt'
testTagetFile = '/content/drive/MyDrive/Colab Notebooks/SpamData/03_Testing/test-target-file.txt'

vocabSize = 2500

##Read and load features from the .txt files into NumPy Array


In [7]:
sparseTrain = np.loadtxt(trainingDataFile, delimiter=' ', dtype=int)

In [6]:
sparseTest = np.loadtxt(testingDataFile, delimiter=' ', dtype=int)

In [12]:
print('No of rows in the training data:', sparseTrain.shape[0])
print('No of rows in the testing data:', sparseTest.shape[0])
print('No of emails in the training data:', np.unique(sparseTrain[:, 0]).size)
print('No of emails in the testing data:', np.unique(sparseTest[:, 0]).size)

No of rows in the training data: 260990
No of rows in the testing data: 114874
No of emails in the training data: 4022
No of emails in the testing data: 1717


##Creating an empty DataFrame

In [17]:
colNames = ['DOC_ID'] + ['CATEGORY'] + list(range(0, vocabSize))
indexNames = np.unique(sparseTrain[:, 0])

In [20]:
trainDataEmp = pd.DataFrame(index = indexNames, columns = colNames)
trainDataEmp.fillna(value=0, inplace=True)
trainDataEmp

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Creating a Full Matrix from a Sparse Matrix

In [19]:
def makeFullMatrix(sparseMatrix, nrWords, docID=0, wordID=1, catID=2, freq=3):
    
    colNames = ['DOC_ID'] + ['CATEGORY'] + list(range(0, vocabSize))
    docIDNames = np.unique(sparseMatrix[:, 0])
    
    fullMatrix = pd.DataFrame(index = docIDNames, columns = colNames)
    fullMatrix.fillna(value=0, inplace=True)

    for i in range(sparseMatrix.shape[0]):
        docNR = sparseMatrix[i][docID]
        wordIDX = sparseMatrix[i][wordID]
        label = sparseMatrix[i][catID]
        occurance = sparseMatrix[i][freq]

        fullMatrix.at[docNR, 'DOC_ID'] = docNR
        fullMatrix.at[docNR, 'CATEGORY'] = label
        fullMatrix.at[docNR, wordIDX] = occurance
    
    fullMatrix.set_index('DOC_ID', inplace=True)
    return fullMatrix

In [21]:
%%time

trainData = makeFullMatrix(sparseTrain, vocabSize)

CPU times: user 11.6 s, sys: 128 ms, total: 11.7 s
Wall time: 11.6 s


In [22]:
trainData

,CATEGORY,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,2,4,2,0,0,4,0,0,2,0,3,0,0,2,0,10,2,2,2,3,0,0,2,0,7,0,2,4,2,6,0,0,0,8,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
1,1,3,3,0,0,1,0,0,3,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,0,3,0,0,0,0,0,0,4,2,0,0,0,2,0,0,0,2,0,2,0,0,0,0,2,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,2,1,0,0,3,0,0,0,1,0,0,0,1,0,0,2,2,0,0,0,1,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5790,0,1,0,1,0,1,1,0,0,3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5791,0,6,4,0,0,3,0,6,2,11,6,0,13,4,8,1,12,1,1,0,0,3,0,1,2,7,1,0,2,1,4,2,0,8,28,11,1,4,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


##Training the Naive Bayes Model

###Calculating the overall probability of spam emails

In [24]:
probSpam = trainData.CATEGORY.sum() / trainData.CATEGORY.size
print('The probability of SPAM emails is:', probSpam)

The probability of SPAM emails is: 0.3125310790651417


###Calculating the total number of tokens/words in our Data Set

In [26]:
trainDataFeatures = trainData.loc[:, trainData.columns != 'CATEGORY']
trainDataFeatures

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,4,2,0,0,4,0,0,2,0,3,0,0,2,0,10,2,2,2,3,0,0,2,0,7,0,2,4,2,6,0,0,0,8,0,0,0,0,2,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
1,3,3,0,0,1,0,0,3,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,3,0,0,0,0,0,0,4,2,0,0,0,2,0,0,0,2,0,2,0,0,0,0,2,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,2,1,0,0,3,0,0,0,1,0,0,0,1,0,0,2,2,0,0,0,1,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5790,1,0,1,0,1,1,0,0,3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5791,6,4,0,0,3,0,6,2,11,6,0,13,4,8,1,12,1,1,0,0,3,0,1,2,7,1,0,2,1,4,2,0,8,28,11,1,4,0,0,4,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [33]:
emailLengths = trainDataFeatures.sum(axis=1)
totalWordCount = emailLengths.sum()
print('The total number of words in the training dataset is:', totalWordCount)

The total number of words in the training dataset is: 438636


###Number of Words in Spam and Non Spam Emails

In [41]:
spamLenght = emailLengths[trainData.CATEGORY == 1]
notSpamLenght = emailLengths[trainData.CATEGORY == 0]

spamWC = spamLenght.sum()
notSpamWC = notSpamLenght.sum()

print('The total number of words in spam emails is:', spamWC)
print('The total number of words in non-spam emails is:', notSpamWC)

The total number of words in spam emails is: 188693
The total number of words in non-spam emails is: 249943


###Summing the tokens occuring in Spam

In [42]:
trainSpamTokens = trainDataFeatures.loc[trainData.CATEGORY == 1]
trainSpamTokens

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,4,2,0,0,4,0,0,2,0,3,0,0,2,0,10,2,2,2,3,0,0,2,0,7,0,2,4,2,6,0,0,0,8,0,0,0,0,2,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
1,3,3,0,0,1,0,0,3,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,3,0,0,0,0,0,0,4,2,0,0,0,2,0,0,0,2,0,2,0,0,0,0,2,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,2,1,0,0,3,0,0,0,1,0,0,0,1,0,0,2,2,0,0,0,1,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1887,2,8,2,0,2,6,2,0,1,0,3,0,0,0,0,2,4,2,2,0,5,0,2,2,0,0,2,0,2,0,0,0,0,4,0,0,2,0,2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
1889,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
#This is known as LaPlace Smooting wherein we add one to the sets where the data is predefined as zero, in order not to encounter anything divided by 0
summedSpamTokens = trainSpamTokens.sum(axis=0) + 1 
summedSpamTokens

0       2029
1        896
2       1264
3       2087
4       1213
        ... 
2495      34
2496      15
2497      31
2498       1
2499      45
Length: 2500, dtype: int64

###Summing the  tokens occuring in non spam emails

In [48]:
trainNotSpamTokens = trainDataFeatures.loc[trainData.CATEGORY == 0]
trainNotSpamTokens

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1896,3,0,1,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1898,1,2,0,0,0,0,1,0,0,0,0,2,0,2,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1899,2,4,0,2,0,0,0,0,1,0,0,2,1,0,1,0,1,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,3,0,0,0,2,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1900,3,0,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1901,8,1,3,3,2,2,1,2,0,0,1,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5790,1,0,1,0,1,1,0,0,3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5791,6,4,0,0,3,0,6,2,11,6,0,13,4,8,1,12,1,1,0,0,3,0,1,2,7,1,0,2,1,4,2,0,8,28,11,1,4,0,0,4,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [49]:
#This is known as LaPlace Smooting wherein we add one to the sets where the data is predefined as zero, in order not to encounter anything divided by 0
summedNotSpamTokens = trainNotSpamTokens.sum(axis=0) + 1 
summedNotSpamTokens

0       5271
1       2588
2       2019
3        913
4       1630
        ... 
2495       6
2496      16
2497       2
2498      39
2499       1
Length: 2500, dtype: int64

##P(Token | Spam): Probability that a Token occurs given the email is spam 

In [52]:
probTokensSpam = summedSpamTokens / (spamWC + vocabSize)
probTokensSpam

0       0.010612
1       0.004686
2       0.006611
3       0.010916
4       0.006344
          ...   
2495    0.000178
2496    0.000078
2497    0.000162
2498    0.000005
2499    0.000235
Length: 2500, dtype: float64

##P(Token | Not Spam): Probability that a Token occurs given the email is not spam 

In [55]:
probTokensNotSpam = summedNotSpamTokens / (notSpamWC + vocabSize)
probTokensNotSpam

0       0.020880
1       0.010252
2       0.007998
3       0.003617
4       0.006457
          ...   
2495    0.000024
2496    0.000063
2497    0.000008
2498    0.000154
2499    0.000004
Length: 2500, dtype: float64

##P(Token): the overall probability that a token occurs

In [63]:
probTokensAll = trainDataFeatures.sum(axis=0) / (totalWordCount)
probTokensAll

0       0.016638
1       0.007938
2       0.007480
3       0.006835
4       0.006477
          ...   
2495    0.000087
2496    0.000066
2497    0.000071
2498    0.000087
2499    0.000100
Length: 2500, dtype: float64

##Saving the trained model

In [64]:
np.savetxt(tokenSpamProbFile, probTokensSpam)
np.savetxt(tokenNotSpamProbFile, probTokensNotSpam)
np.savetxt(tokenAllProbFile, probTokensAll)

##Prepare the Test Data

In [65]:
%%time

testData = makeFullMatrix(sparseTest, vocabSize)

CPU times: user 6.05 s, sys: 63.6 ms, total: 6.11 s
Wall time: 6.09 s


In [66]:
testData

,CATEGORY,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,7,1,0,1,0,0,0,1,0,2,2,0,0,0,3,0,1,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,1,2,0,2,0,0,4,3,0,0,0,3,0,0,0,0,2,4,8,2,0,1,0,2,0,4,0,2,0,2,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,1,0,0,0,1,2,4,2,3,1,1,1,0,1,1,4,0,1,0,2,0,2,2,4,3,0,0,1,0,0,0,2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,3,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,0,3,0,2,0,0,2,1,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5786,0,0,7,1,0,0,2,11,1,3,1,3,3,3,0,1,2,0,0,1,0,0,8,1,2,0,0,0,0,0,2,3,0,0,0,1,2,4,1,4,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5788,0,7,3,4,3,3,8,2,0,3,0,4,0,2,2,1,1,1,0,1,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
xTest = testData.loc[:, testData.columns != 'CATEGORY']
yTest = testData.CATEGORY

In [70]:
np.savetxt(testFeaturesMatrixFile, xTest)
np.savetxt(testTagetFile, yTest)